In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/epileptic-seizure-recognition/Epileptic Seizure Recognition.csv


In [3]:
df = pd.read_csv("/kaggle/input/epileptic-seizure-recognition/Epileptic Seizure Recognition.csv")

In [4]:
df

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,...,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,...,121,135,148,143,116,86,68,59,55,3


In [7]:
labels = df['y']
labels = pd.get_dummies(labels,  prefix = 'y')

In [8]:
labels

,y_1,y_2,y_3,y_4,y_5
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
11495,0,1,0,0,0
11496,1,0,0,0,0
11497,0,0,0,0,1
11498,0,0,1,0,0


In [10]:
df = df.drop(['Unnamed', 'y'], axis = 1)

In [11]:
arr = df.iloc[:,:].values

In [12]:
coef_arr = []
freqs_arr = []

In [13]:
import pywt
scales = np.arange(1, 41)

for i in range(11500):
    coef, freqs = pywt.cwt(arr[i], scales, 'cgau8')
    coef_arr.append(coef)
    freqs_arr.append(freqs)

In [17]:
import matplotlib.pyplot as plt
# Ploting Scalogram

plt.figure(figsize = (8, 8))
plt.imshow(abs(coef_arr[0]), interpolation = 'bilinear', cmap = 'hsv', aspect = 'auto', vmax = abs(coef_arr[0]).max(), 
           vmin = -abs(coef_arr[0]).max())
plt.gca().invert_yaxis()
plt.close()

In [15]:
os.makedirs('/kaggle/working/scalograms')

In [24]:
my_dpi = 130
dirs = ['s', 'f', 'n', 'o', 'z']

for i in range(10000, 11500):
    plt.figure(figsize=(64/my_dpi, 64/my_dpi), dpi=my_dpi)
    plt.imshow(abs(coef_arr[i]), interpolation = 'bilinear', cmap='hsv', aspect = 'auto', vmax = abs(coef_arr[i]).max(), 
               vmin = -abs(coef_arr[i]).max())
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.savefig(f'/kaggle/working/scalograms/img{i}.png', bbox_inches='tight', pad_inches=0.0)
    plt.close()

In [25]:
sg_img = []

In [26]:
import cv2

for i in range(11500):
 
    image = cv2.imread(f'/kaggle/working/scalograms/img{i}.png')

    # BGR -> GRAY
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # let's resize our image to be 150 pixels wide, but in order to
    # prevent our resized image from being skewed/distorted, we must
    # first calculate the ratio of the *new* width to the *old* width
#     r_1 = 32 / img.shape[0]
#     r_2 = 128 / img.shape[1]
    dim = (32, 32)
    # perform the actual resizing of the image
    img = cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('s_img1.png', img)
    sg_img.append(img)

In [27]:
sg_img[0].shape

(32, 32)

In [30]:
#sg_df = pd.DataFrame()
for i in range(9297, 11500):
    temp_df = pd.DataFrame(sg_img[i].flatten())
    sg_df = pd.concat([sg_df, temp_df], axis = 1, ignore_index=True)

In [31]:
sg_df

,0,1,2,3,4,5,6,7,8,9,...,11490,11491,11492,11493,11494,11495,11496,11497,11498,11499
0,71,34,83,82,58,57,99,33,111,58,...,88,91,38,83,100,50,113,75,34,95
1,78,46,83,82,59,57,97,47,110,69,...,84,86,40,81,95,47,114,80,33,97
2,83,60,81,86,53,53,94,64,110,77,...,81,83,43,81,92,41,118,82,35,104
3,87,71,72,92,46,47,94,76,113,83,...,81,82,45,83,91,33,118,85,40,114
4,94,83,58,100,35,36,94,90,113,89,...,86,81,45,87,91,39,119,85,46,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,178,178,178,178,176,178,178,178,178,178,...,178,178,178,178,176,178,178,178,178,178
1020,178,178,178,178,178,178,178,178,178,178,...,178,178,178,178,176,178,177,176,178,178
1021,178,178,178,175,178,178,178,178,178,178,...,178,178,178,178,177,178,178,173,178,178
1022,178,178,178,178,178,178,178,178,175,177,...,178,178,178,178,178,178,178,171,178,177


In [32]:
sg_df = sg_df.T

In [33]:
sg_df

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,71,78,83,87,94,98,98,97,95,94,...,178,178,178,178,178,178,178,178,178,178
1,34,46,60,71,83,97,103,99,95,92,...,178,176,178,178,178,178,178,178,178,178
2,83,83,81,72,58,44,33,60,89,120,...,178,178,178,178,178,178,178,178,178,177
3,82,82,86,92,100,99,81,65,49,35,...,176,178,177,176,178,178,178,175,178,177
4,58,59,53,46,35,35,46,62,73,80,...,177,176,178,178,178,176,178,178,178,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,50,47,41,33,39,47,46,39,33,37,...,178,178,178,178,178,178,178,178,178,178
11496,113,114,118,118,119,121,125,125,125,121,...,178,178,178,178,178,178,177,178,178,178
11497,75,80,82,85,85,79,64,45,39,79,...,176,174,173,163,176,178,176,173,171,170
11498,34,33,35,40,46,50,53,55,55,60,...,178,178,178,178,178,178,178,178,178,178


In [34]:
sg_df.columns = [f"Pixel{i}" for i in range(1024)]

In [35]:
sg_df

,Pixel0,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,Pixel9,...,Pixel1014,Pixel1015,Pixel1016,Pixel1017,Pixel1018,Pixel1019,Pixel1020,Pixel1021,Pixel1022,Pixel1023
0,71,78,83,87,94,98,98,97,95,94,...,178,178,178,178,178,178,178,178,178,178
1,34,46,60,71,83,97,103,99,95,92,...,178,176,178,178,178,178,178,178,178,178
2,83,83,81,72,58,44,33,60,89,120,...,178,178,178,178,178,178,178,178,178,177
3,82,82,86,92,100,99,81,65,49,35,...,176,178,177,176,178,178,178,175,178,177
4,58,59,53,46,35,35,46,62,73,80,...,177,176,178,178,178,176,178,178,178,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,50,47,41,33,39,47,46,39,33,37,...,178,178,178,178,178,178,178,178,178,178
11496,113,114,118,118,119,121,125,125,125,121,...,178,178,178,178,178,178,177,178,178,178
11497,75,80,82,85,85,79,64,45,39,79,...,176,174,173,163,176,178,176,173,171,170
11498,34,33,35,40,46,50,53,55,55,60,...,178,178,178,178,178,178,178,178,178,178


In [36]:
combined_df = pd.concat([sg_df, labels], axis = 1)

In [37]:
combined_df

,Pixel0,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,Pixel9,...,Pixel1019,Pixel1020,Pixel1021,Pixel1022,Pixel1023,y_1,y_2,y_3,y_4,y_5
0,71,78,83,87,94,98,98,97,95,94,...,178,178,178,178,178,0,0,0,1,0
1,34,46,60,71,83,97,103,99,95,92,...,178,178,178,178,178,1,0,0,0,0
2,83,83,81,72,58,44,33,60,89,120,...,178,178,178,178,177,0,0,0,0,1
3,82,82,86,92,100,99,81,65,49,35,...,178,178,175,178,177,0,0,0,0,1
4,58,59,53,46,35,35,46,62,73,80,...,176,178,178,178,178,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,50,47,41,33,39,47,46,39,33,37,...,178,178,178,178,178,0,1,0,0,0
11496,113,114,118,118,119,121,125,125,125,121,...,178,177,178,178,178,1,0,0,0,0
11497,75,80,82,85,85,79,64,45,39,79,...,178,176,173,171,170,0,0,0,0,1
11498,34,33,35,40,46,50,53,55,55,60,...,178,178,178,178,178,0,0,1,0,0


In [38]:
combined_df.to_csv("Scalogram_Data.csv")